In [4]:
from src.utils import *
from src.utils import numericalSort
import imageio

def load_images(fnames):
    d_list = []
    for fname in fnames[:13]:
        image_raw = imageio.read(fname)
        image = np.array(image_raw.get_data(0))
        d_list.append(image)
    return d_list


def load_dataset():
    import glob

    # List of file names of actual Satellite images for traininig
    filelist_trainx = sorted(glob.glob('/Users/laszlofreund/code/ai/satellite-image-segmentation/data/The-Eye-in-the-Sky-dataset/sat/*.tif'), key=numericalSort)
    # List of file names of classified images for traininig
    filelist_trainy = sorted(glob.glob('/Users/laszlofreund/code/ai/satellite-image-segmentation/data/The-Eye-in-the-Sky-dataset/gt/*.tif'), key=numericalSort)
    # List of file names of actual Satellite images for testing
    filelist_testx = sorted(glob.glob('/Users/laszlofreund/code/ai/satellite-image-segmentation/data/The-Eye-in-the-Sky-test-data/sat_test/*.tif'), key=numericalSort)

    # Making array of all the training sat images as it is without any cropping
    x_train = load_images(filelist_trainx[:13])
    ytrain = load_images(filelist_trainy[:13])
    x_test = load_images(filelist_testx)

    x_val = x_train[:-1]
    y_val = ytrain[-1]

    return x_train, ytrain, x_val, y_val, x_test


In [5]:
from src.model import UNet
import torch
from torch.utils.data import Dataset, DataLoader

x_train, y_train, x_val, y_val, x_test = load_dataset()


class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.transpose(torch.tensor(self.x_data[idx].astype('uint8'), dtype=torch.float), 0, 2)
        y = torch.transpose(torch.tensor(self.y_data[idx].astype('uint8'), dtype=torch.float), 0, 2)
        return x, y

# Assuming x_train, y_train, x_val, y_val, x_test are NumPy arrays
train_dataset = CustomDataset(x_train, y_train)
val_dataset = CustomDataset(x_val, y_val)
test_dataset = CustomDataset(x_test, torch.zeros(len(x_test)))  # Placeholder for test, since y_test is not available

# Define batch size for DataLoader
batch_size = 1

# Create DataLoader for training, validation, and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
unet = UNet()
unet.train_model(train_loader, val_loader, num_epochs=10)

RuntimeError: MPS backend out of memory (MPS allocated: 14.78 GB, other allocations: 3.16 GB, max allowed: 18.13 GB). Tried to allocate 319.16 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [8]:
x_val[0].shape

(1295, 1364, 4)